<a href="https://colab.research.google.com/github/HanzhouLiu/Deep-Learning-with-Python-Exercises/blob/main/BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Design, train and test a neural network model for the IMDB dataset, using bidirectional LSTM
from __future__ import print_function
import numpy as np

from keras.preprocessing import sequence
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.datasets import imdb

In [2]:
max_features = 2000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 100

In [4]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


25000 train sequences
25000 test sequences


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [5]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

Pad sequences (samples x time)
x_train shape: (25000, 100)
x_test shape: (25000, 100)


In [6]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [7]:
opti_ = Adam(lr=0.0025)
model.compile(optimizer=opti_, loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 128)          256000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               98816     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 354,945
Trainable params: 354,945
Non-trainable params: 0
_________________________________________________________________


In [9]:
batch_size = 64
epochs = 20

In [10]:
from time import time

In [11]:
t1=time()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=[x_test, y_test])
t2=time()

Epoch 1/20
391/391 [==============================] - 41s 20ms/step - loss: 0.5200 - accuracy: 0.7205 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
391/391 [==============================] - 7s 18ms/step - loss: 0.3314 - accuracy: 0.8585 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/20
391/391 [==============================] - 7s 17ms/step - loss: 0.2736 - accuracy: 0.8877 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/20
391/391 [==============================] - 7s 17ms/step - loss: 0.2319 - accuracy: 0.9070 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/20
391/391 [==============================] - 7s 17ms/step - loss: 0.1868 - accuracy: 0.9270 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/20
391/391 [==============================] - 7s 18ms/step - loss: 0.1495 - accuracy: 0.9429 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/20
391/391 [==============================] - 7s 17ms/step - loss: 0.1182 - accuracy:

In [12]:
t_delta = round((t2-t1)/60,3)
print(f"{epochs} took total {t_delta} minutes.")

20 took total 2.849 minutes.


In [13]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 0.005
    if epoch >= 3:
        lr *= 0.5
    if epoch >= 7:
        lr *= 0.25
    if epoch >= 11:
        lr *= 0.5
    if epoch >= 16:
        lr *= 0.5
        
    print('Learning rate: ', lr)
    return lr

In [14]:
lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
callbacks = [lr_reducer, lr_scheduler]
#callbacks = [lr_scheduler]

model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
opti_ = Adam(lr=lr_schedule(0))
model.compile(optimizer=opti_, loss='binary_crossentropy', metrics=['accuracy'])

t1=time()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=[x_test, y_test],
          callbacks=callbacks)
t2=time()

Learning rate:  0.005
Epoch 1/20
Learning rate:  0.005
391/391 [==============================] - 10s 19ms/step - loss: 0.5378 - accuracy: 0.7065 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
Learning rate:  0.005
391/391 [==============================] - 7s 17ms/step - loss: 0.3663 - accuracy: 0.8414 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/20
Learning rate:  0.005
391/391 [==============================] - 7s 17ms/step - loss: 0.3150 - accuracy: 0.8675 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/20
Learning rate:  0.0025
391/391 [==============================] - 7s 18ms/step - loss: 0.2333 - accuracy: 0.9039 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/20
Learning rate:  0.0025
391/391 [==============================] - 7s 18ms/step - loss: 0.1892 - accuracy: 0.9245 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/20
Learning rate:  0.0025
391/391 [==============================] - 7s 17ms/step - loss: 0.1442 - acc